In [1]:
import numpy as np
import xarray

In [2]:
fname = '/g/data/w40/clv563/ozflux_data/Samford_2022_L3.nc'

In [ ]:
latent_heat_flux_name = 'Fe'
sensible_heat_flux_name = 'Fh_PFP'

In AUS2200 files, sensible heat flux is hfss and latent heat flux is hfls

In [3]:
ds = xarray.open_dataset(fname)

latent_hfx_obs = ds.Fe
sensible_hfx_obs = ds.Fh_PFP

FileNotFoundError: [Errno 2] No such file or directory: b'/g/data/w40/clv563/ozflux_data/Samford_2022_L3.nc'